In [ ]:
import random

def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

def remove_duplicates(problem):
    unique = set()
    for el in problem:
        unique.add(frozenset(el))
    return unique    

def goal(N):
    return set(range(N))

def test(l, goal):
    return set(l) == goal

## Evaluation
The `score` function evaluates the fitness of an individual, favoring coverage of the problem and penalizing repetitions.

In [ ]:
def genome_to_single_list(g, problem):
    selection = list()
    for s, p in zip(g, problem):
        if s:
            selection.append(p)
    return [item for subl in selection for item in subl]

def score(g, problem):
    selection = genome_to_single_list(g, problem)
    unique_len = len(set(selection))
    nrep = len(selection) - unique_len
    return unique_len - nrep/(1 + unique_len)

## Individual
An `Individual` is represented by the tuple (`genome`, `fitness`), respectively a list of [1,0] and a float.

In [ ]:
from collections import namedtuple

Individual = namedtuple("Individual", ["genome", "fitness"])

In [ ]:
def generate_population(problem_size, population_size):
    population = list()
    for genome in [tuple([random.choice([1, 0]) for _ in range(problem_size)]) for _ in range(population_size)]:
        population.append(Individual(genome, score(genome)))

## Mutation
`mutate` inverts the value of one random locus of the genome of an individual.

In [ ]:
def mutate(i, problem):
    g = i.genome
    point = random.randint(0, len(g) - 1)
    new_g = g[:point] + (1 - g[point],) + g[point + 1 :]
    return Individual(new_g, score(new_g, problem))

## Crossing-over
`crossover` replaces a random portion of the genome of the first individual with the corresponding part of the genome of the second individual.

In [ ]:
def cross_over(i1, i2, problem):
    g1 = i1.genome
    g2 = i2.genome
    cut = random.randint(0, len(g1))
    g = g1[:cut] + g2[cut:]
    return Individual(g, score(g, problem))

## Tournament
`tournament` selects the `2` (parameter `tournament_size`) best individuals from the population, based on their fitness.

In [ ]:
def tournament(population, tournament_size=2):
    return max(random.choices(population, k=tournament_size), key=lambda i: i.fitness)

In [ ]:
def evolution(N, Nproblem, generations, pop_size, off_size):
    
    population = list()
    old_best = Individual([], 0)
    stagnation_counter = 0
    
    for genome in [tuple([random.choice([1, 0]) for _ in range(len(Nproblem))]) for _ in range(pop_size)]:
        population.append(Individual(genome, score(genome, Nproblem)))

    for g in range(generations):
        
        if (stagnation_counter > generations/10):
            break

        offspring = list()
        for i in range(off_size):
            if random.random() < 0.10:
                p = tournament(population)
                o = mutate(p, Nproblem)
            else:
                p1 = tournament(population)
                p2 = tournament(population)
                o = cross_over(p1, p2, Nproblem)
            offspring.append(o)
        population += offspring
        population = sorted(population, key=lambda i: i.fitness, reverse=True)[:pop_size]

        if (population[0] == old_best):
            stagnation_counter += 1
        else:
            old_best = population[0]
            stagnation_counter = 0
    
    for i in population:
        if (test(genome_to_single_list(i.genome, Nproblem), goal(N))):
            return i
    print("↓↓↓ NOT ACCEPTABLE! ↓↓↓")
    return population[0]

In [ ]:
NUM_GENERATIONS = 500
INITIAL_POPULATION_SIZE = 1000
SEED = 42

for size in [5, 10, 20, 100, 500, 1000]:
    population_size = int(INITIAL_POPULATION_SIZE)
    offspring_size = int(population_size)
    Nproblem = remove_duplicates(problem(size, seed=SEED))
    best = evolution(size, Nproblem, NUM_GENERATIONS, population_size, offspring_size)
    print("Size: ", size, "\tSolution: ", len(genome_to_single_list(best.genome, Nproblem)))